# Overlapとmp_term_nameの統合

- 例：abnormal_circulating_glucose_level（連続値）
- 例：male_infertility（カテゴリ値）

In [2]:
# Move up to top directory
import os
os.getcwd()
os.chdir('../')
os.getcwd()

'/mnt/g/impc-genes_with_similar_phenotypes'

In [7]:
from pathlib import Path
from collections import defaultdict
from itertools import combinations
import csv
import pandas as pd
from matplotlib import pyplot as plt

In [8]:
path_overlap = Path("data", "overlap_ratios.csv")
df_overlap = pd.read_csv(path_overlap)

In [9]:
# path_mp = Path("data", "mp_term_name", "decreased_circulating_glucose_level.csv")
path_mp = Path("data", "mp_term_name", "male_infertility.csv")
df_mp = pd.read_csv(path_mp)
df_mp = df_mp[["marker_symbol", "p_value", "effect_size"]].drop_duplicates()


In [10]:
print(df_overlap.head(3))

print(len(df_mp))

df_mp

  marker1 marker2  overlap_ratio  overlapped_mp_number  \
0    Dpf2   Gna13          0.167                     2   
1    Dpf2    Plec          0.200                     2   
2    Dpf2  Gnptab          0.080                     2   

                                       overlapped_mp  \
0  abnormal embryo size\npreweaning lethality, co...   
1   edema\npreweaning lethality, complete penetrance   
2  abnormal heart morphology\nabnormal kidney mor...   

                                          marker1_mp  \
0  abnormal craniofacial morphology\nabnormal emb...   
1  abnormal craniofacial morphology\nabnormal emb...   
2  abnormal craniofacial morphology\nabnormal emb...   

                                          marker2_mp  
0  abnormal embryo size\nabnormal visceral yolk s...  
1   edema\npreweaning lethality, complete penetrance  
2  abnormal blood vessel morphology\nabnormal hea...  
318


,marker_symbol,p_value,effect_size
0,Tex38,0,1
1,Pou2af3,0,1
2,Stoml2,0,1
3,Sbf1,0,1
4,Cyrib,0,1
...,...,...,...
320,Cep19,0,1
321,Bzw1,0,1
322,Septin10,0,1
323,Cfap57,0,1


In [11]:
print(df_mp["effect_size"].describe())

count    318.0
mean       1.0
std        0.0
min        1.0
25%        1.0
50%        1.0
75%        1.0
max        1.0
Name: effect_size, dtype: float64


In [12]:
# Absolute value of effect size

df_mp.loc[:, "effect_size"] = df_mp["effect_size"].abs()

In [13]:
# print(df_overlap[df_overlap["intersect_count"] > 5 & df_overlap["overlap_ratio"] > 0.5])

In [14]:
# df_overlap_filterd = df_overlap[df_overlap["overlapped_mp_number"] > 0]
df_overlap_filterd = df_overlap[(df_overlap["overlap_ratio"] > 0.3) & (df_overlap["overlapped_mp_number"] > 2)]

In [15]:
df_filtered = df_overlap_filterd[
    (df_overlap_filterd['marker1'].isin(df_mp['marker_symbol'])) &
    (df_overlap_filterd['marker2'].isin(df_mp['marker_symbol']))
]

In [16]:
df_merged = pd.merge(df_filtered, df_mp, how='inner', left_on='marker1', right_on='marker_symbol')

print(len(df_merged))

df_merged.head(3)

208


,marker1,marker2,overlap_ratio,overlapped_mp_number,overlapped_mp,marker1_mp,marker2_mp,marker_symbol,p_value,effect_size
0,Camsap3,Kash5,0.348,8,abnormal bone structure\ndecreased bone minera...,abnormal auditory brainstem response\nabnormal...,abnormal bone structure\ndecreased bone minera...,Camsap3,0,1
1,Camsap3,Herc1,0.310,9,abnormal auditory brainstem response\nabnormal...,abnormal auditory brainstem response\nabnormal...,abnormal auditory brainstem response\nabnormal...,Camsap3,0,1
2,Camsap3,Clpp,0.303,10,abnormal auditory brainstem response\nabnormal...,abnormal auditory brainstem response\nabnormal...,abnormal auditory brainstem response\nabnormal...,Camsap3,0,1


### 出力

In [17]:
df_network = df_merged[["marker1", "marker2", "effect_size", "overlap_ratio", "marker1_mp", "marker2_mp", "overlapped_mp"]]

df_network.columns = ["from", "to", "node_size", "edge_size", "from_mp", "to_mp", "edge_mp"]
print(len(df_network))
print(df_network.head(3))


208
      from     to  node_size  edge_size  \
0  Camsap3  Kash5          1      0.348   
1  Camsap3  Herc1          1      0.310   
2  Camsap3   Clpp          1      0.303   

                                             from_mp  \
0  abnormal auditory brainstem response\nabnormal...   
1  abnormal auditory brainstem response\nabnormal...   
2  abnormal auditory brainstem response\nabnormal...   

                                               to_mp  \
0  abnormal bone structure\ndecreased bone minera...   
1  abnormal auditory brainstem response\nabnormal...   
2  abnormal auditory brainstem response\nabnormal...   

                                             edge_mp  
0  abnormal bone structure\ndecreased bone minera...  
1  abnormal auditory brainstem response\nabnormal...  
2  abnormal auditory brainstem response\nabnormal...  


In [18]:
df_network.to_csv(Path("data", "network.csv"), index=False)

In [41]:
# %%bash

# python scripts/vis_network.py